# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

J. Neumann  ->  J. Neumann  |  ['J. Neumann']
C. Mao  ->  C. Mao  |  ['C. Mao']
E. Schinnerer  ->  E. Schinnerer  |  ['E. Schinnerer']
G. Chauvin  ->  G. Chauvin  |  ['G. Chauvin']
T. Henning  ->  T. Henning  |  ['T. Henning']
G.-D. Marleau  ->  G.-D. Marleau  |  ['G.-D. Marleau']
M. Samland  ->  M. Samland  |  ['M. Samland']


K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']
Arxiv has 53 new papers today
          3 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/3 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2409.18180
extracting tarball to tmp_2409.18180...

 done.


Found 170 bibliographic references in tmp_2409.18180/main.bbl.
Issues with the citations
syntax error in line 1166: '=' expected
Retrieving document from  https://arxiv.org/e-print/2409.18793
extracting tarball to tmp_2409.18793...

 done.


...

Found 94 bibliographic references in tmp_2409.18793/main.bbl.
Retrieving document from  https://arxiv.org/e-print/2409.18882


extracting tarball to tmp_2409.18882...

 done.
  0: tmp_2409.18882/AandA.tex, 1,041 lines
  1: tmp_2409.18882/old_main.tex, 1,534 lines
  3: tmp_2409.18882/main.tex, 2,169 lines


/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 3 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.20/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2409.18882/main.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


Found 39 bibliographic references in tmp_2409.18882/main.bbl.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.18180-b31b1b.svg)](https://arxiv.org/abs/2409.18180) | **Azimuthal Variations of Stellar Populations in Barred Galaxies**  |
|| <mark>J. Neumann</mark>, et al. -- incl., <mark>C. Mao</mark>, <mark>E. Schinnerer</mark> |
|*Appeared on*| *2024-10-01*|
|*Comments*| *18 pages, 12 figures (+appendix), accepted for publication in MNRAS*|
|**Abstract**|            Bars are expected to impact the distribution of stellar populations both during bar formation, as they rearrange stars into new orbits, and afterwards, due to the redistribution of star-formation-fuelling gas and transfer of angular momentum. We study the impact of stellar bars on the azimuthal variation of stellar population age, metallicity and mass surface density in $\sim1\,000$ nearby barred galaxies from the SDSS-IV/MaNGA survey. Bars have higher stellar mass density ($0.113^{+0.065}_{-0.067}$ dex) and are more metal-rich ($0.028^{+0.033}_{-0.040}$ dex) than the discs at the same radii. Stellar ages show a variety of bar to inter-bar contrasts with no consistent trend. The difference in metallicity increases with total stellar mass of the galaxy and distance below the star-forming main sequence. We discuss a combination of potentially responsible processes including kinematic separation, more extended star formation histories and more efficient recycling in bars and at bar-spiral arm connections. Additionally, we observe an offset (10°-40°) of the peak metallicity to the bar major axis in star-forming bars in low-mass galaxies, and more metal-rich regions outside the ends of the bar in long bars and quenched galaxies. Furthermore, there is a subtle trend of lower metallicities on the leading side of spiral arms compared to the trailing side. Finally, we report a spiral arm surface density feature, which could point towards a dominant bar-spiral connection and pitch angle of $\alpha \sim 25$°. We interpret these features in the context of bar formation and the impact of large-scale gas flows associated with their presence.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.18793-b31b1b.svg)](https://arxiv.org/abs/2409.18793) | **Giant planets population around B stars from the first part of the BEAST survey**  |
|| P. Delorme, et al. -- incl., <mark>G. Chauvin</mark>, <mark>T. Henning</mark>, <mark>G.-D. Marleau</mark>, <mark>M. Samland</mark> |
|*Appeared on*| *2024-10-01*|
|*Comments*| *Accepted in A&A*|
|**Abstract**|            Exoplanets form from circumstellar protoplanetary discs whose fundamental properties (notably their extent, composition, mass, temperature and lifetime) depend on the host star properties, such as their mass and luminosity. B-stars are among the most massive stars and their protoplanetary discs test extreme conditions for exoplanet formation. This paper investigates the frequency of giant planet companions around young B-stars (median age of 16 Myr) in the Scorpius-Centaurus association, the closest association containing a large population of B-stars. We systematically search for massive exoplanets with the high-contrast direct imaging instrument SPHERE using the data from the BEAST survey, that targets an homogeneous sample of young B-stars from the wide Sco-Cen association. We derive accurate detection limits in case of non-detections. We found evidence in previous papers for two substellar companions around 42 stars. The masses of these companions are straddling the ~13 Jupiter mass deuterium burning limit but their mass ratio with respect to their host star is close to that of Jupiter. We derive a frequency of such massive planetary mass companions around B stars of 11-5+7%, accounting for the survey sensitivity. The discoveries of substellar companions bcen b and mu2sco B happened after only few stars in the survey had been observed, raising the possibility that massive Jovian planets might be common around B-stars. However our statistical analysis show that the occurrence rate of such planets is similar around B-stars and around solar-type stars of similar age, while B-star companions exhibit low mass ratios and larger semi-major axis.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2409.18882-b31b1b.svg)](https://arxiv.org/abs/2409.18882) | **Euclid preparation: 6x2 pt analysis of Euclid's spectroscopic and photometric data sets**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2024-10-01*|
|*Comments*| *32 pages, 20 figures. Comments are welcome*|
|**Abstract**|            We present cosmological parameter forecasts for the Euclid 6x2pt statistics, which include the galaxy clustering and weak lensing main probes together with previously neglected cross-covariance and cross-correlation signals between imaging/photometric and spectroscopic data. The aim is understanding the impact of such terms on the Euclid performance. We produce 6x2pt cosmological forecasts, considering two different techniques: the so-called harmonic and hybrid approaches, respectively. In the first, we treat all the different Euclid probes in the same way, i.e. we consider only angular 2pt-statistics for spectroscopic and photometric clustering, as well as for weak lensing, analysing all their possible cross-covariances and cross-correlations in the spherical harmonic domain. In the second, we do not account for negligible cross-covariances between the 3D and 2D data, but consider the combination of their cross-correlation with the auto-correlation signals. We find that both cross-covariances and cross-correlation signals, have a negligible impact on the cosmological parameter constraints and, therefore, on the Euclid performance. In the case of the hybrid approach, we attribute this result to the effect of the cross-correlation between weak lensing and photometric data, which is dominant with respect to other cross-correlation signals. In the case of the 2D harmonic approach, we attribute this result to two main theoretical limitations of the 2D projected statistics implemented in this work according to the analysis of official Euclid forecasts: the high shot noise and the limited redshift range of the spectroscopic sample, together with the loss of radial information from subleading terms such as redshift-space distortions and lensing magnification. Our analysis suggests that 2D and 3D Euclid data can be safely treated as independent, with a great saving in computational resources.         |

## Failed papers

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2409.18180/./Figures/SFMS_10.png', 'tmp_2409.18180/./Figures/MILES_hm_rot.png', 'tmp_2409.18180/./Figures/violins_v7_fullsample_MILES.png']
copying  tmp_2409.18180/./Figures/SFMS_10.png to _build/html/
copying  tmp_2409.18180/./Figures/MILES_hm_rot.png to _build/html/
copying  tmp_2409.18180/./Figures/violins_v7_fullsample_MILES.png to _build/html/
exported in  _build/html/2409.18180.md
    + _build/html/tmp_2409.18180/./Figures/SFMS_10.png
    + _build/html/tmp_2409.18180/./Figures/MILES_hm_rot.png
    + _build/html/tmp_2409.18180/./Figures/violins_v7_fullsample_MILES.png
found figures ['tmp_2409.18793/./figs/CMD_BEAST_BCKG_K12.png', 'tmp_2409.18793/./figs/mean_ames_cond_detmaps.png', 'tmp_2409.18793/./figs/all_mean_detmaps.png']
copying  tmp_2409.18793/./figs/CMD_BEAST_BCKG_K12.png to _build/html/
copying  tmp_2409.18793/./figs/mean_ames_cond_detmaps.png to _build/html/
copying  tmp_2409.18793/./figs/all_mean_detmaps.png to _build/html/
exported in  _build/html/24

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\ff}{{\sc firefly}}$
$\newcommand{\ffvac}{\texttt{MaNGA {\sc firefly} VAC}}$
$\newcommand{\arraystretch}{1.3}$
$\newcommand{\thebibliography}{\DeclareRobustCommand{\VAN}[3]{##3}\VANthebibliography}$</div>



<div id="title">

# Azimuthal Variations of Stellar Populations in Barred Galaxies

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.18180-b31b1b.svg)](https://arxiv.org/abs/2409.18180)<mark>Appeared on: 2024-10-01</mark> -  _18 pages, 12 figures (+appendix), accepted for publication in MNRAS_

</div>
<div id="authors">

<mark>J. Neumann</mark>, et al. -- incl., <mark>C. Mao</mark>, <mark>E. Schinnerer</mark>

</div>
<div id="abstract">

**Abstract:** Bars are expected to impact the distribution of stellar populations both during bar formation, as they rearrange stars into new orbits, and afterwards, due to the redistribution of star-formation-fuelling gas and transfer of angular momentum. We study the impact of stellar bars on the azimuthal variation of stellar population age, metallicity and mass surface density in $\sim1 000$ nearby barred galaxies from the SDSS-IV/MaNGA survey. Bars have higher stellar mass density ( $0.113^{+0.065}_{-0.067}$ dex) and are more metal-rich ( $0.028^{+0.033}_{-0.040}$ dex) than the discs at the same radii. Stellar ages show a variety of bar to inter-bar contrasts with no consistent trend. The difference in metallicity increases with total stellar mass of the galaxy and distance below the star-forming main sequence. We discuss a combination of potentially responsible processes including kinematic separation, more extended star formation histories and more efficient recycling in bars and at bar-spiral arm connections. Additionally, we observe an offset ( $10◦$ - $40◦$ ) of the peak metallicity to the bar major axis in star-forming bars in low-mass galaxies, and more metal-rich regions outside the ends of the bar in long bars and quenched galaxies. Furthermore, there is a subtle trend of lower metallicities on the leading side of spiral arms compared to the trailing side. Finally, we report a spiral arm surface density feature, which could point towards a dominant bar-spiral connection and pitch angle of $\alpha \sim 25◦$ . We interpret these features in the context of bar formation and the impact of large-scale gas flows associated with their presence.

</div>

<div id="div_fig1">

<img src="tmp_2409.18180/./Figures/SFMS_10.png" alt="Fig1" width="100%"/>

**Figure 1. -** Sample representation in total SFR versus total stellar mass plane. SFRs are based on dust-corrected H$\alpha$ measurements from \texttt{Pipe3D}\citep{Sanchez2022}. Points are coloured by scaled bar length with L$_\mathrm{bar}$ as described in Sect. \ref{sect:lbar} and R$_{90}$ from the enhanced NASA Sloan Atlas \citep{Wake2017}. Star-forming bars are marked using the information provided in \citet{FraserMcKelvie2020a}. The black solid line marks the star-forming main sequence (SFMS) as determined by \citet{Renzini2015} and the dotted line 0.5 dex below the solid line represents our selection of galaxies on or below the main sequence. The two dashed vertical lines illustrate our subsamples separated in mass and the two white horizontal lines on the colourbar show our sample separation by bar length. Points outside the y-axis range are illustrated as arrows at the bottom. (*fig:sample*)

</div>
<div id="div_fig2">

<img src="tmp_2409.18180/./Figures/MILES_hm_rot.png" alt="Fig4" width="100%"/>

**Figure 4. -** Azimuthal variations of stellar populations for all high-mass galaxies with determinable sense of rotation, showing, from top to bottom, light-weighted age, [Z/H] and $\Sigma_\star$. Left panels show medians per polar grid cell, while the right panels show significance maps of these medians. The black solid circle marks the bar radius and the dashed circle marks the average 2$\times$FWHM of the PSF across the subsample. The dotted line follows a log-spiral function with a pitch angle of $ 25.2◦$. Zero degree is towards the right, 90 degree towards the top. The arrow indicates the anti-clockwise rotation of the galaxies. (*fig:polar_example*)

</div>
<div id="div_fig3">

<img src="tmp_2409.18180/./Figures/violins_v7_fullsample_MILES.png" alt="Fig11" width="100%"/>

**Figure 11. -** Bar-to-inter-bar contrast of age, [Z/H] and $\Sigma_\star$ for all subsamples discussed in the text. Violins show the distribution of individual galaxies. Marked are the medians (white dots), interquartile ranges (vertical grey bars) and standard errors on the medians (black error bars). (*fig:violins*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.18180"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\eg}{{\it e.g.}, }$
$\newcommand{\ie}{{\it i.e.}, }$
$\newcommand{\ms}{m s^{\rm -1}}$
$\newcommand{\kms}{\ensuremath{\mathrm{km} \mathrm{s}^{-1}}}$
$\newcommand{\mearth}{M_{\rm Earth}}$
$\newcommand{\vsini}{v\sin{i}}$
$\newcommand{\elodie}{E{\small LODIE}}$
$\newcommand{\sophie}{S{\small OPHIE}}$
$\newcommand{\harps}{H{\small ARPS}}$
$\newcommand{\thetacyg}{\theta Cygni}$
$\newcommand{\bp}{\beta Pictoris }$
$\newcommand{\bpic}{\beta Pictoris }$
$\newcommand{\mds}{\mu ^2~Sco }$
$\newcommand{\bcen}{b Centauri }$
$\newcommand$
$\newcommand$
$\newcommand{◦ee}{^{\circ}}$
$\newcommand{\arraystretch}{1.5}$
$\newcommand{\DvtwoD}{\ensuremath{v_{\mathrm{2D}}}\xspace}$
$\newcommand{\DvthreeD}{\ensuremath{v_{\mathrm{3D}}}\xspace}$
$\newcommand{\pcapt}{\ensuremath{p_{\mathrm{capt}}\xspace}}$
$\newcommand{\rmin}{\ensuremath{r_{\mathrm{min}}\xspace}}$</div>



<div id="title">

# Giant planets population around B stars from the first part of the BEAST survey$\thanks{Based on data obtained with the ESO/VLT SPHERE instrument under programs 1101.C-0258(A/B/C/D).}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.18793-b31b1b.svg)](https://arxiv.org/abs/2409.18793)<mark>Appeared on: 2024-10-01</mark> -  _Accepted in A&A_

</div>
<div id="authors">

P. Delorme, et al. -- incl., <mark>G. Chauvin</mark>, <mark>T. Henning</mark>, <mark>G.-D. Marleau</mark>, <mark>M. Samland</mark>

</div>
<div id="abstract">

**Abstract:** Exoplanets form from circumstellar protoplanetary discs whose fundamental properties (notably their extent, composition, mass, temperature and lifetime) depend on the host star properties, such as their mass and luminosity. B-stars are among the most massive stars and their protoplanetary discs test extreme conditions for exoplanet formation. This paper investigates the frequency of giant planet companions around young B-stars (median age of 16 Myr) in the Scorpius-Centaurus association, the closest association containing a large population of B-stars. We systematically search for massive exoplanets with the high-contrast direct imaging instrument SPHERE using the data from the BEAST survey, that targets an homogeneous sample of young B-stars from the wide Sco-Cen association. We derive accurate detection limits in case of non-detections. We found evidence in previous papers for two substellar companions around 42 stars. The masses of these companions are straddling the $\sim$ 13 Jupiter mass deuterium burning limit but their mass ratio with respect to their host star is close to that of Jupiter. We derive a frequency of such massive planetary mass companions around B stars of $11_{-5}^{+7}$ \% , accounting for the survey sensitivity. The discoveries of substellar companions $\bcen$ b and $\mds$ B happened after only few stars in the survey had been observed, raising the possibility that massive Jovian planets might be common around B-stars. However our statistical analysis show that the occurrence rate of such planets is similar around B-stars and around solar-type stars of similar age, while B-star companions exhibit low mass ratios and larger semi-major axis.

</div>

<div id="div_fig1">

<img src="tmp_2409.18793/./figs/CMD_BEAST_BCKG_K12.png" alt="Fig2" width="100%"/>

**Figure 2. -** K1-K2 color-magnitude diagram of candidate companions found around the original, 42-strong, sample. Detected sources are represented by the hollow dots while the plain symbols represent the theoretical MLT track \citep[][Appendix C]{Bonnefoy_GJ504_2018}. (*fig:CMD*)

</div>
<div id="div_fig2">

<img src="tmp_2409.18793/./figs/mean_ames_cond_detmaps.png" alt="Fig4" width="100%"/>

**Figure 4. -** Average detection probability of a companion of given mass/sma in the BEAST intermediary survey using the cond atmospheric model. The two detected planets in this survey are also represented with the mass / sma extracted from [Janson, Gratton and Rodet (2021)]() and [Squicciarini, et. al (2021)](). Jupiter is also added to provide comparison. In the case of $\bcen$ b, no semi-major axis estimation is available, so we represented its projected separation. (*fig:sample_detmaps*)

</div>
<div id="div_fig3">

<img src="tmp_2409.18793/./figs/all_mean_detmaps.png" alt="Fig9" width="100%"/>

**Figure 9. -** Mean probability detection maps computed using 3 different model and the nominal, minimum and maximum age for all stars indicated in Table \ref{tab:star_table}. (*fig:all_mean_detmaps*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.18793"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand$
$\newcommand$
$\newcommand{\onesigma}{1\mbox{--}\sigma}$
$\newcommand{\Omb}{\Omega_\mathrm{b}}$
$\newcommand{\Omm}{\Omega_\mathrm{m}}$
$\newcommand{\mnu}{M_\nu}$
$\newcommand{\GCph}{\mathrm{GC}_\mathrm{ph}}$
$\newcommand{\GCsp}{\mathrm{GC}_\mathrm{sp}}$
$\newcommand{\WL}{\mathrm{WL}}$
$\newcommand{\wz}{w_0}$
$\newcommand{\wa}{w_a}$
$\newcommand{\sige}{\sigma_{8}}$
$\newcommand{\FoM}{\mathrm{FoM}}$
$\newcommand{\LCDM}{\Lambda\mathrm{CDM}}$
$\newcommand{\wzwaCDM}{\wz\wa\mathrm{CDM}}$
$\newcommand{\bivz}{b_{\mathrm{v},i}(z)}$
$\newcommand{\bv}{b_\mathrm{v}}$
$\newcommand{\bveff}{b_{\mathrm{v}\mathrm{eff}}}$
$\newcommand{\nivz}{n_i^\textrm{v}(z)}$
$\newcommand{\vis}{VIS}$
$\newcommand{\nisp}{NISP}$
$\newcommand{\pkz}{P_{\mathrm{mm}}(k,z)}$
$\newcommand{\pmkz}{P_{\mathrm{mm}}(k,z)}$
$\newcommand{\pmlimb}{P_{\textrm{mm}}\left[\frac{\ell+1 / 2}{r(z)}, z\right]}$
$\newcommand{\cl}[3]{C^{#1}_{#2}({#3})}$
$\newcommand{\cnol}[2]{C^{#1}_{#2}}$
$\newcommand{\Halpha}{\mathrm{H}\alpha}$
$\newcommand{\de}{\mathrm{d}}$
$\newcommand{\clest}{\hat{C}(\ell)}$
$\newcommand{\clgg}{C^{\gamma\gamma}(\ell)}$
$\newcommand{\clABij}{C^\mathrm{AB}_{ij}(\ell)}$
$\newcommand{\clestABij}{\hat{C}^{\mathrm{AB}}_{ij}(\ell)}$
$\newcommand{\clggij}{C^{\gamma \gamma}_{ij}(\ell)}$
$\newcommand{\lmax}{\ell_{\mathrm{max}}}$
$\newcommand{\lmin}{\ell_{\min}}$
$\newcommand{\fisher}{F_{\alpha \beta}}$
$\newcommand{\CAMB}{\texttt{CAMB}}$
$\newcommand{\camb}{\texttt{CAMB}}$
$\newcommand{\lcdm}{\Lambda \mathrm{CDM}}$
$\newcommand{\nlcdm}{\nu \lcdm}$
$\newcommand{\wcdm}{w_0 w_a \mathrm{CDM}}$
$\newcommand{\nwcdm}{\nu \wcdm}$
$\newcommand{\Hzero}{H_0}$
$\newcommand{\HzeroUNIT}{\text{km}   \text{s}^{-1}   \text{Mpc}^{-1}}$
$\newcommand{\HzeroVAL}{100   \HzeroUNIT}$
$\newcommand{\Hz}{H(z)}$
$\newcommand{\Ommhh}{\Omm h^2}$
$\newcommand{\omb}{\omega_\textrm{b}}$
$\newcommand{\Ombhh}{\Omb h^2}$
$\newcommand{\omc}{\omega_\textrm{c}}$
$\newcommand{\Omc}{\Omega_\textrm{c}}$
$\newcommand{\Omchh}{\Omc h^2}$
$\newcommand{\Omn}{\Omega_\nu}$
$\newcommand{\Omnhh}{\Omn h^2}$
$\newcommand{\OmL}{\Omega_\Lambda}$
$\newcommand{\Mnu}{M_\nu}$
$\newcommand{\ns}{n_\textrm{s}}$
$\newcommand{\As}{A_\textrm{s}}$
$\newcommand{\sigeSCALE}{8 h^{-1} \mbox{Mpc}}$
$\newcommand{\refMnu}{0.06}$
$\newcommand{\refMnueV}{\refMnu   \mbox{eV}}$
$\newcommand{\wklens}{\mathrm{WL}}$
$\newcommand{\bigblock}[1]{\vec{\mathcal{#1}}(\ell)}$
$\newcommand{\tomoblock}[1]{\mathbf{#1}(\ell)}$
$\newcommand{\deltac}{\delta_\mathrm{c}}$
$\newcommand{\deltav}{\delta_\mathrm{v}}$
$\newcommand{\pzpz}{p_{\mathrm{ph}}\left(z_{\mathrm{p}} | z\right)}$
$\newcommand{\ngz}{n^\mathrm{g}(z)}$
$\newcommand{\nigz}{n_i^\mathrm{g}(z)}$
$\newcommand{\aIA}{\mathcal{A}_{\mathrm{IA}}}$
$\newcommand{\betaIA}{\beta_{\mathrm{IA}}}$
$\newcommand{\etaIA}{\eta_{\mathrm{IA}}}$
$\newcommand{\CIA}{C_{\mathrm{IA}}}$
$\newcommand{\wlwl}{\mathrm{wlwl}}$
$\newcommand{\phph}{\mathrm{phph}}$
$\newcommand{\spsp}{\mathrm{spsp}}$
$\newcommand{\wlph}{\mathrm{wlph}}$
$\newcommand{\phwl}{\mathrm{phwl}}$
$\newcommand{\wlsp}{\mathrm{wlsp}}$
$\newcommand{\spwl}{\mathrm{spwl}}$
$\newcommand{\phsp}{\mathrm{phsp}}$
$\newcommand{\spph}{\mathrm{spph}}$
$\newcommand{\xc}[2]{\mathrm{XC}(#1,#2)}$
$\newcommand{\threetwoptShort}{\ensuremath{\textrm{3\times2pt}}}$
$\newcommand{\threetwoptLong}{\WL+\GCph+\xc{\WL}{\GCph}}$
$\newcommand{\sixtwoptShort}{\ensuremath{\textrm{6\times2pt}}}$
$\newcommand{\sixtwoptLong}{\WL+\GCph+\GCsp+\xc{\WL}{\GCph}+\xc{\WL}{\GCsp}+\xc{\GCph}{\GCsp}}$
$\newcommand{\positive}[1]{\textcolor{ForestGreen}{#1}}$
$\newcommand{\negative}[1]{\textcolor{red}{#1}}$
$\newcommand{ç}[1]{{\textcolor{magenta}{#1}}}$
$\newcommand{\bcl}[2]{\mathbf{C}^{#1}(#2)}$
$\newcommand{\hatbcl}[1]{\hat{\mathbf{C}}^{#1}(\ell)}$
$\newcommand{\sgl}[3]{\Sigma^{#1}_{#2}(#3)}$
$\newcommand{\bsgl}[2]{\mathbf{\Sigma}^{#1}(#2)}$
$\newcommand{\cov}[2]{\operatorname{Cov}\left[#1, #2\right]}$
$\newcommand{\invcov}[2]{\operatorname{Cov}^{-1}\left[#1, #2\right]}$
$\newcommand{\pdv}[2]{\frac{\partial #1}{\partial #2}}$
$\newcommand{\orcid}[1]$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$</div>



<div id="title">

# $\Euclid$ preparation: 6$\times$2pt analysis of $\Euclid$'s spectroscopic and photometric data sets

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2409.18882-b31b1b.svg)](https://arxiv.org/abs/2409.18882)<mark>Appeared on: 2024-10-01</mark> -  _32 pages, 20 figures. Comments are welcome_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:**            We present cosmological parameter forecasts for the Euclid 6x2pt statistics, which include the galaxy clustering and weak lensing main probes together with previously neglected cross-covariance and cross-correlation signals between imaging/photometric and spectroscopic data. The aim is understanding the impact of such terms on the Euclid performance. We produce 6x2pt cosmological forecasts, considering two different techniques: the so-called harmonic and hybrid approaches, respectively. In the first, we treat all the different Euclid probes in the same way, i.e. we consider only angular 2pt-statistics for spectroscopic and photometric clustering, as well as for weak lensing, analysing all their possible cross-covariances and cross-correlations in the spherical harmonic domain. In the second, we do not account for negligible cross-covariances between the 3D and 2D data, but consider the combination of their cross-correlation with the auto-correlation signals. We find that both cross-covariances and cross-correlation signals, have a negligible impact on the cosmological parameter constraints and, therefore, on the Euclid performance. In the case of the hybrid approach, we attribute this result to the effect of the cross-correlation between weak lensing and photometric data, which is dominant with respect to other cross-correlation signals. In the case of the 2D harmonic approach, we attribute this result to two main theoretical limitations of the 2D projected statistics implemented in this work according to the analysis of official Euclid forecasts: the high shot noise and the limited redshift range of the spectroscopic sample, together with the loss of radial information from subleading terms such as redshift-space distortions and lensing magnification. Our analysis suggests that 2D and 3D Euclid data can be safely treated as independent, with a great saving in computational resources.         

</div>

<div id="div_fig1">

<img src="tmp_2409.18882/./Figures/6x2pt/6x2pt_cov_impact.png" alt="Fig14" width="100%"/>

**Figure 14. -** Impact on FoM and marginalised 1-$\sigma$ uncertainties, of cross covariances between $\GCsp$ and $\threetwoptShort  ([$\WL$+$\GCph$+$\xc${$\WL$}{$\GCph$}])$, quantified with percentage differences on the constraints, in the optimistic (left) and pessimistic (right) scenarios. The reference for the percentage are the constraints of $[$\threetwoptShort$] + [$\GCsp$]$, where $\GCsp$ is considered as *independent* from the rest. Note the opposite sign of the percentage differences for FoM and uncertainties. (*fig:3x2pt_sp_cov_impact*)

</div>
<div id="div_fig2">

<img src="tmp_2409.18882/./Figures/weight_functions_plot.png" alt="Fig1" width="100%"/>

**Figure 1. -** Weight functions for the three probes considered in the forecast. For $\WL$ at high redshift bins, the weight function \cref{eq:weak_lensing_full_weight}(solid line) becomes negative, due to the contribution of intrinsic alignment (IA). The shear weight function \cref{eq:shear_weight}(dashed line) remains instead always positive as it should be. (*fig:weight_functions_plot*)

</div>
<div id="div_fig3">

<img src="tmp_2409.18882/./Figures/6x2pt/6x2pt_harmonic_xcs_impact_ref_3x2ptxGCsp.png" alt="Fig15" width="100%"/>

**Figure 15. -** Impact of the $\xc{$\GCph$}{$\GCsp$}$ and $\xc{$\WL$}{$\GCsp$}$ cross-correlations on marginalised $\onesigma$ uncertainties, with respect to the constraints given by the $[$\threetwoptShort$+$\GCsp$]$ Fisher matrix, in which the cross-covariance between $\threetwoptShort$ statistics and $\GCsp$ is taken into account. The top panels refer to the optimistic scenario, while the bottom panels refer to the pessimistic one. The percentage differences related to $\xc{$\WL$}{$\GCsp$}$ have been multiplied by $100$ to make them visible when using a single scale on the $y$ axis. (*fig:6x2pt_harmonic_xcs_impact_ref_3x2pt_gcsp_cov*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2409.18882"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

203  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

14  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
